In [9]:
# modeling.ipynb – FINAL IMPROVED VERSION with fast NUTS sampler support
# Goal: Bayesian single change point detection – faster sampling

# Cell 1 – Imports (including optional fast samplers)
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pymc as pm
import arviz as az

# ── Fast NUTS samplers (install at least one of them)
try:
    import numpyro
    NUTS_SAMPLER_AVAILABLE = "numpyro"
    print("numpyro detected → fast NUTS sampler available")
except ImportError:
    try:
        import blackjax
        NUTS_SAMPLER_AVAILABLE = "blackjax"
        print("blackjax detected → fast NUTS sampler available")
    except ImportError:
        NUTS_SAMPLER_AVAILABLE = None
        print("Warning: neither numpyro nor blackjax found")
        print("→ sampling will be much slower (use default PyMC NUTS)")
        print("Install one of them for 5–15× speedup:")
        print("   pip install numpyro   # recommended")
        print("or pip install blackjax")

from src.data_preprocessor import BrentDataPreprocessor

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

numpyro detected → fast NUTS sampler available


In [10]:
# Cell 2 – Load & prepare data (expects focus period 2012+)
print("Loading data...")

preprocessor = BrentDataPreprocessor()
df_full = preprocessor.add_features(focus_period=None)
df = preprocessor.get_processed(focus_period='2012-01-01')

prices = df['Price'].values.astype(np.float64)
dates  = df.index
N      = len(prices)

print(f"Period: {dates.min():%Y-%m-%d} → {dates.max():%Y-%m-%d}")
print(f"Observations: {N:,}")
print(f"Mean price: ${prices.mean():.2f}    Std: ${prices.std():.2f}")

Loading data...
Raw data loaded: 9011 rows, 2 columns
Data loaded successfully: 1987-05-20 to 2022-11-14
Total observations: 9011
Subset to >= 2012-01-01: 2760 observations remain
Period: 2012-01-03 → 2022-11-14
Observations: 2,760
Mean price: $74.14    Std: $26.66


In [ ]:
# Cell 3 – Safe version + silence warning
import pytensor
pytensor.config.cxx = ""   # ← Add this line

print("Building and sampling the model...")

with pm.Model() as model:
    tau = pm.DiscreteUniform('tau', lower=0, upper=N-1)
    mu1 = pm.Normal('mu1', mu=prices.mean(), sigma=prices.std()*2.5)
    mu2 = pm.Normal('mu2', mu=prices.mean(), sigma=prices.std()*2.5)
    sigma = pm.HalfNormal('sigma', sigma=prices.std()*1.8)
    mu_t = pm.math.switch(tau >= np.arange(N), mu2, mu1)
    y = pm.Normal('y', mu=mu_t, sigma=sigma, observed=prices)

    trace = pm.sample(
        draws=500,
        tune=1000,
        chains=2,
        target_accept=0.90,
        random_seed=42,
        return_inferencedata=True,
        progressbar=True
    )

print("Sampling finished.")

Building and sampling the model...


Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>Metropolis: [tau]
>NUTS: [mu1, mu2, sigma]
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


Output()

ValueError: Not enough samples to build a trace.